<a href="https://colab.research.google.com/github/dlscks/Jimat/blob/master/%EB%A9%94%EB%89%B4_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
+from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/seoulfood

Mounted at /content/drive
/content/drive/MyDrive/seoulfood


In [ ]:
!pip install selenium
!pip install -U  urllib3 requests

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# 구역 csv파일 읽어오기기
import pandas as pd
pd.read_csv('seoulfood/영등포구url.csv', encoding='cp949', sep=",")

,url,맛집명,타입,주소,영업시간,전화번호,대표사진주소,평점,X,Y,법정동명
0,https://place.map.kakao.com/7851288,또순이네,"육류,고기",서울 영등포구 선유로47길 16,영업시간 매일 11:30 ~ 22:00,02-2672-2255,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,3.1,126.895391,37.535870,양평동4가
1,https://place.map.kakao.com/745164647,코끼리베이글,"제과,베이커리",서울 영등포구 선유로 176 수화공단 1층 60호,영업시간 매일 08:30 ~ 20:00,070-4184-0082,NaN,3.0,126.893139,37.529029,양평동3가
2,https://place.map.kakao.com/12048202,대한옥,설렁탕,서울 영등포구 영등포로51길 6,영업시간 월~토 11:00 ~ 21:00,02-2633-5052,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,3.3,126.910883,37.519354,영등포동2가
3,https://place.map.kakao.com/10546066,송죽장,중국요리,서울 영등포구 문래로 203,영업시간 매일 11:00 ~ 22:00 · 브레이크타임 매일 15:30 ~ 17:00,02-2678-1323,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,2.6,126.904712,37.518252,영등포동4가
4,https://place.map.kakao.com/339122660,카페1953위드오드리,카페,서울 영등포구 경인로77가길 12 1층,영업시간 매일 10:00 ~ 22:00,02-2672-0504,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.0,126.892147,37.511954,문래동2가
...,...,...,...,...,...,...,...,...,...,...,...
495,https://place.map.kakao.com/16878314,막내회쎈타,회,서울 영등포구 도신로53길 7,영업시간 매일 10:30 ~ 22:00,02-842-6150,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,3.6,126.914408,37.511908,신길동
496,https://place.map.kakao.com/8095989,청하,회,서울 영등포구 국제금융로 78 408호,영업시간 월~토 11:30 ~ 22:30 · 브레이크타임 월~토 15:00 ~ 17:30,02-785-7721,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,5.0,126.931315,37.520874,여의도동
497,https://place.map.kakao.com/2006926319,빨강떡볶이 영등포점,떡볶이,서울 영등포구 영중로3길 10,영업시간 월~금 11:00 ~ 20:30,02-719-3449,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.0,126.905141,37.516778,영등포동4가
498,https://place.map.kakao.com/1318530005,신촌황소곱창 영등포직영점,"곱창,막창",서울 영등포구 영중로4길 15 1층,매일 14:00 ~ 02:00,NaN,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,2.0,126.908183,37.517572,영등포동3가


In [ ]:
import os
import re
from time import sleep
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium.webdriver.common.by import By

df = pd.read_csv('seoulfood/중랑구url.csv', encoding='cp949', sep=",")

df_url = df['url']

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

#컬럼 생성
menuNameSearch = []
menuPriceSearch = []
store_data = {}

#크롤링
for url in df_url :
  driver.get(url)
  time.sleep(1)

  try :
      menuNameSearchElement = driver.find_elements(By.CLASS_NAME, 'loss_word')

      menuNameSearchElementLIst = []
      
      print(menuNameSearchElementLIst)

      for roll in range(len(menuNameSearchElement)) :
        menuNameSearchElementLIst.append(menuNameSearchElement[roll].text)

      menuNameSearch.append(menuNameSearchElementLIst)

  except NoSuchElementException:
      menuNameSearch.append(None)
      # print("메뉴가 존재하지 않습니다.")
  
  try :
      menuPriceSearchElement = driver.find_elements(By.CLASS_NAME, 'price_menu')
      menuPriceSearchElementList = []
      for roll in range(len(menuPriceSearchElement)) :
        menuPriceSearchElementList.append(menuPriceSearchElement[roll].text)
      menuPriceSearch.append(menuPriceSearchElementList)

  except NoSuchElementException:  
      menuPriceSearch.append(None)

store_data = {'menuNameSearch' : menuNameSearch, 'menuPriceSearch' : menuPriceSearch}

WebDriverException: ignored

In [ ]:
# 크롤링한 데이터 저장
result = pd.DataFrame(store_data)

df['menuNameSearch'] = result['menuNameSearch']
df['menuPriceSearch'] = result['menuPriceSearch']

df.to_csv("seoulfood/중랑구url1.csv", encoding='cp949')

In [ ]:
# 크롤링한 데이터 저장한거 읽기기
pd.read_csv("seoulfood/중랑구url1.csv", encoding='cp949')

,Unnamed: 0,url,맛집명,타입,주소,영업시간,전화번호,대표사진주소,평점,X,Y,법정동명,menuNameSearch,menuPriceSearch
0,0,https://place.map.kakao.com/16704031,용마해장국,해장국,서울 중랑구 용마공원로5길 8,영업시간 화~일 10:00 ~ 20:00 · 브레이크타임 화~일 14:00 ~ 15:00,02-2209-5938,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.3,127.101547,37.594457,망우동,['해장국'],"['8,000']"
1,1,https://place.map.kakao.com/20545292,한국횟집,회,서울 중랑구 중랑역로 34,영업시간 매일 14:00 ~ 02:00,02-439-3331,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,3.8,127.076217,37.597082,중화동,"['광어 (소)', '자연산 놀래미 (소)', '광어+생연어 (소)', '생연어초밥...","['40,000', '60,000', '50,000', '15,000', '35,0..."
2,2,https://place.map.kakao.com/1685526961,소담촌 상봉점,샤브샤브,서울 중랑구 망우로 328 시네마시티 15층 1501호,영업시간 매일 11:00 ~ 22:00,02-434-1999,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,2.7,127.088960,37.596829,상봉동,"['월남쌈 샤브 정식', '월남쌈 버섯 샤브 정식', '명품한우 샤브 정식', '와...","['10,900', '11,800', '15,000', '14,000', '16,0..."
3,3,https://place.map.kakao.com/27205199,쭈꾸미킹 상봉점,"해물,생선",서울 중랑구 송림길 13,영업시간 매일 11:00 ~ 22:30,02-439-0035,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,3.5,127.095388,37.599969,상봉동,"['쭈꾸미볶음', '철판쭈꾸미', '철판삼겹쭈꾸미', '왕새우튀김 (4ea)', '...","['9,000', '13,000', '14,000', '8,000', '9,000']"
4,4,https://place.map.kakao.com/16415084,이주소곱창,"곱창,막창",서울 중랑구 봉우재로33길 34,NaN,02-978-4723,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.0,127.087998,37.593956,상봉동,"['곱창구이', '양구이', '대창구이', '막창구이', '곱창전골 (중)']","['25,000', '27,000', '24,000', '24,000', '32,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,https://place.map.kakao.com/440367256,한국통닭,치킨,서울 중랑구 망우로 367,NaN,02-435-0719,NaN,4.8,127.093164,37.598474,상봉동,"['3마리', '2마리', '1마리', '닭똥집', '감자튀김']","['10,000', '7,000', '5,000', '4,000', '4,000']"
496,496,https://place.map.kakao.com/2027468895,메가MGC커피 면목중앙점,커피전문점,서울 중랑구 동일로 612 1층,NaN,02-2207-7858,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,2.8,127.080138,37.584894,면목동,"['아메리카노', '카푸치노', '꿀아메리카노', '민트카페모카', '메가리카노',...",[]
497,497,https://place.map.kakao.com/1816271809,생활맥주 중랑망우점,"호프,요리주점",서울 중랑구 용마산로115길 39 1층,영업시간 매일 18:00 ~ 03:00,02-434-8122,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.7,127.098167,37.598603,망우동,"['생활 부대 떡볶이', '크리스피 텐더와 새우', '앵그리버드', '앵그리버드RE...",[]
498,498,https://place.map.kakao.com/1005926308,서원 망우점,중국요리,서울 중랑구 용마산로 441,영업시간 매일 10:00 ~ 22:00,02-2208-1013,//t1.kakaocdn.net/thumb/T800x0.q80/?fname=http...,4.2,127.097321,37.590856,망우동,"['인절미 미니 탕수육', '양장피 (소)', '크림새우 (소)', '동파육', '...","['8,000', '25,000', '25,000', '40,000', '13,00..."
